# Workflow to calculate the quantum efficiency of a polycrystalline material

In [2]:
from ase.calculators.castep import Castep
from ase.atoms import Atoms

from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.wulff import WulffShape

from functions import calc_surface_energy
from functions import get_qe

ImportError: cannot import name 'cal_surface_energy' from 'functions' (c:\Users\Felix Mildner\Desktop\University\London - Imperial\PhD\Project\coding\photoemission\functions.py)

I recommend using pymatgen to generate the bulk/slab and then convert it to an ASE atom object. The advantages of this approach are:
- pymatgen has a better bulk/slab generation algorithm 
- castep can link a calculator to the Atom object


#### Generate the bulk structure

#### Generate the surface structure

#### Run the calculations

#### Extract the final energies

#### Calculate the surface energy

#### Get the % of each surface from a  Wulff construction

#### Launch OptaDOS

#### Get QE from OptaDOS output

#### Calculate the weighted average QE

#### Calculate and display plenty more properties (DOS, bands, ...)